In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/SRCNN_Pytorch

/content/drive/MyDrive/Colab Notebooks/SRCNN_Pytorch


In [ ]:
!git clone https://github.com/kymikim0401/SRCNN_Pytorch

Cloning into 'SRCNN_Pytorch'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 23 (delta 1), reused 19 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 469.20 KiB | 14.22 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
!python train.py

===> Epoch 1 of 50
Train PSNR: 11.322, Val PSNR 14.525, time: 129.06
===> Epoch 2 of 50
Train PSNR: 12.636, Val PSNR 14.762, time: 144.77
===> Epoch 3 of 50
Train PSNR: 12.686, Val PSNR 14.767, time: 160.76
===> Epoch 4 of 50
Train PSNR: 12.701, Val PSNR 14.761, time: 176.17
===> Epoch 5 of 50
Train PSNR: 12.695, Val PSNR 14.770, time: 191.78
===> Epoch 6 of 50
Train PSNR: 12.705, Val PSNR 14.777, time: 207.55
===> Epoch 7 of 50
Train PSNR: 12.709, Val PSNR 14.784, time: 223.08
===> Epoch 8 of 50
Train PSNR: 12.713, Val PSNR 14.788, time: 238.68
===> Epoch 9 of 50
Train PSNR: 12.716, Val PSNR 14.795, time: 254.81
===> Epoch 10 of 50
Train PSNR: 12.720, Val PSNR 14.798, time: 270.19
===> Epoch 11 of 50
Train PSNR: 12.723, Val PSNR 14.802, time: 285.78
===> Epoch 12 of 50
Train PSNR: 12.725, Val PSNR 14.804, time: 301.37
===> Epoch 13 of 50
Train PSNR: 12.727, Val PSNR 14.806, time: 317.45
===> Epoch 14 of 50
Train PSNR: 12.728, Val PSNR 14.808, time: 333.82
===> Epoch 15 of 50
Train PSN

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/SRCNN_Pytorch

/content/drive/MyDrive/Colab Notebooks/SRCNN_Pytorch


In [ ]:
!python dataset.py

In [ ]:
!python test.py

102061.png | PSNR: 22.30 dB | SSIM: 0.6733
109053.png | PSNR: 24.42 dB | SSIM: 0.6496
12084.png | PSNR: 22.36 dB | SSIM: 0.5081
157055.png | PSNR: 21.47 dB | SSIM: 0.5669
170057.png | PSNR: 24.50 dB | SSIM: 0.6480
19021.png | PSNR: 21.66 dB | SSIM: 0.5251
196073.png | PSNR: 25.35 dB | SSIM: 0.4184
197017.png | PSNR: 23.35 dB | SSIM: 0.5530
219090.png | PSNR: 22.12 dB | SSIM: 0.6058
291000.png | PSNR: 18.52 dB | SSIM: 0.2161
306005.png | PSNR: 22.76 dB | SSIM: 0.5759
3096.png | PSNR: 30.25 dB | SSIM: 0.9188
38082.png | PSNR: 23.54 dB | SSIM: 0.4200
42012.png | PSNR: 24.00 dB | SSIM: 0.6342
43074.png | PSNR: 26.42 dB | SSIM: 0.7309
45096.png | PSNR: 28.41 dB | SSIM: 0.8471
69020.png | PSNR: 23.52 dB | SSIM: 0.3730
85048.png | PSNR: 20.29 dB | SSIM: 0.4397
86016.png | PSNR: 19.83 dB | SSIM: 0.1736
86068.png | PSNR: 24.03 dB | SSIM: 0.4258

Average PSNR: 23.46 dB
Average SSIM: 0.5452


In [ ]:
import cv2
import torch
from torchvision.transforms import ToTensor
from SRCNN_model import SRCNN

def super_resolve_image(path_to_lr_img, model_path='checkpoints/model.pth', scale=4):
    # 1) Read LR BGR image
    lr_bgr = cv2.imread(path_to_lr_img)
    h, w = lr_bgr.shape[:2]

    # 2) Convert to YCrCb and split
    lr_ycrcb = cv2.cvtColor(lr_bgr, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(lr_ycrcb)

    # 3) Upsample each channel to (h*scale, w*scale)
    new_h, new_w = h * scale, w * scale
    y_up  = cv2.resize(y,  (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    cr_up = cv2.resize(cr, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    cb_up = cv2.resize(cb, (new_w, new_h), interpolation=cv2.INTER_CUBIC)

    # 4) Normalize Y and make a [1,1,H,W] tensor
    input_y = ToTensor()(y_up).unsqueeze(0).to("cuda")  # values in [0,1]

    # 5) Load and run SRCNN
    model = SRCNN().to("cuda")
    model.load_state_dict(torch.load(model_path))
    model.eval()
    with torch.no_grad():
        pred_y = model(input_y)             # [1,1,H,W] output
        pred_y = pred_y.squeeze().cpu().numpy()  # [H,W] in [0,1]

    # 6) Denormalize and clip to uint8
    pred_y = (pred_y * 255.0).clip(0,255).astype('uint8')

    # 7) Merge back with the upsampled Cr/Cb
    out_ycrcb = cv2.merge((pred_y, cr_up, cb_up))
    out_bgr   = cv2.cvtColor(out_ycrcb, cv2.COLOR_YCrCb2BGR)

    return out_bgr

# Usage example:
output = super_resolve_image(
    "dataset_img/test/low_res/69020.png",
    model_path="checkpoints/model.pth"
)
cv2.imwrite("output_4x.png", output)